In [ ]:
# Makes plots of S/N vs wavelength and dark current (see s_to_n_logic_life_detectors.py)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u

In [2]:
# constants
# use units of photoelectrons

eta_qm = 0.8 # quantum efficiency
n_int = 1 # number of integrations
t_int = 340000 * u.second # time for 1 integration (sec)
del_Np_prime_del_t = 3.26247387e+01 * u.electron / u.second # Planet signal rate (photo-electrons / sec)       
del_Nstar_prime_del_t = 6.40233542e+06 * u.electron / u.second # Star signal rate (photo-electrons / sec)
null = 0.1 # null transmission of the star
R = 15. * u.electron  # read noise (photo-electrons rms)
n_pix = 6.6 * u.pix # number of pixels under each wavelength element footprint
D = 0.05 * u.electron / (u.pix * u.second) # Dark current rate (photo-electrons / (pix * sec))

In [4]:
def s2n(n_int, t_int, n_pix, del_Np_prime_del_t, del_Nstar_prime_del_t, null, R, D, eta_qm):
    """
    Calculate signal-to-noise ratio with vectorization support.
    
    Any single input variable can be an array while others remain scalars.
    Uses NumPy broadcasting to handle the vectorization automatically.
    """
    
    # Convert all inputs to numpy arrays to enable broadcasting
    # This ensures consistent shapes for vectorized operations
    n_int = np.asarray(n_int)
    t_int = np.asarray(t_int) * u.second
    n_pix = np.asarray(n_pix) * u.pix
    del_Np_prime_del_t = np.asarray(del_Np_prime_del_t) * u.electron / u.second
    del_Nstar_prime_del_t = np.asarray(del_Nstar_prime_del_t) * u.electron / u.second
    null = np.asarray(null)
    R = np.asarray(R) * u.electron
    D = np.asarray(D) * u.electron / (u.pix * u.second)
    eta_qm = np.asarray(eta_qm)

    term_1 = np.sqrt(n_int)

    #print(term_1.value)

    term_2 = eta_qm * t_int * del_Np_prime_del_t

    print('term_2:')
    print(term_2.unit)

    term_3 = eta_qm * t_int * (del_Np_prime_del_t + null * del_Nstar_prime_del_t)

    print('--------------------------------')
    print(del_Np_prime_del_t.unit)
    print((null*del_Nstar_prime_del_t).unit)
    print('term_3:')
    print(term_3.unit)

    term_4 = n_pix * (( R**2/(u.electron * u.pix) ) + t_int * D)

    print('term_4:')
    print(term_4.unit)

    return ( term_1 * term_2 / np.sqrt(term_3 + term_4) ) / u.electron**0.5

In [8]:
test = s2n(n_int=n_int, t_int=t_int, n_pix=n_pix, del_Np_prime_del_t=del_Np_prime_del_t, del_Nstar_prime_del_t=del_Nstar_prime_del_t, null=null, R=R, D=D, eta_qm=eta_qm)

term_2:
electron
--------------------------------
electron / s
electron / s
term_3:
electron
term_4:
electron


In [9]:
test

<Quantity 60.09656687>

In [8]:
# Example usage of vectorized s2n function

# Example 1: Vectorize over dark current values
D_array = np.array([0.1, 1.0, 10.0]) * u.electron / (u.pix * u.second)
s2n_dark_current = s2n(n_int, t_int, n_pix, del_Np_prime_del_t, del_Nstar_prime_del_t, null, R, D_array, eta_qm)
print("S/N for different dark currents:", s2n_dark_current)

# Example 2: Vectorize over integration times
t_int_array = np.array([1, 10, 100]) * u.second
s2n_integration = s2n(n_int, t_int_array, n_pix, del_Np_prime_del_t, del_Nstar_prime_del_t, null, R, D, eta_qm)
print("S/N for different integration times:", s2n_integration)

# Example 3: Vectorize over wavelength (if you have wavelength-dependent parameters)
wavelengths = np.array([4, 6, 8, 10, 12]) * u.um
# Assuming del_Np_prime_del_t varies with wavelength
del_Np_array = np.array([50, 100, 150, 200, 250]) * u.electron / u.second
s2n_wavelength = s2n(n_int, t_int, n_pix, del_Np_array, del_Nstar_prime_del_t, null, R, D, eta_qm)
print("S/N vs wavelength:", s2n_wavelength)


term_2:
electron
--------------------------------
electron / s
electron / s
term_3:
electron
term_4:
electron
S/N for different dark currents: [0.89377386 0.89372366 0.89322215]
term_2:
electron
--------------------------------
electron / s
electron / s
term_3:
electron
term_4:
electron
S/N for different integration times: [0.28189292 0.89322215 2.82518708]
term_2:
electron
--------------------------------
electron / s
electron / s
term_3:
electron
term_4:
electron
S/N vs wavelength: [0.44672247 0.89322215 1.3394993  1.78555416 2.23138697]


In [7]:
test

<Quantity 0.89322215>